In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
answers_df = pd.read_csv('path to answers.csv')

# change unix time to date time
answers_df['time'] = pd.to_datetime(answers_df['time'], unit='ms')
answers_df = answers_df.sort_values(by='time')
answers_df['next_time'] = answers_df['time'].shift(-1)
answerkey_df = pd.read_csv('answer-key.csv')

answerkey_df.rename(columns={'answer': 'correctAnswer'}, inplace=True)

mergedAnswers_df = pd.merge(answers_df, answerkey_df, on=['pageName', 'question'], how='left')
mergedAnswers_df['is_correct'] = mergedAnswers_df['answer'] == mergedAnswers_df['correctAnswer']

mergedAnswers_df = mergedAnswers_df.drop(columns=['next_time'])
mergedAnswers_df = mergedAnswers_df.drop(columns=['time_y'])
mergedAnswers_df = mergedAnswers_df[mergedAnswers_df['version'] != 'undefined']

mergedAnswers_df['penalty_bonus'] = 0

# For special cases in stage 2, apply penalty accordingly
def apply_penalty_bonus(row):
    if row['question'] in ['T1_S2_C1', 'T1_S2_C2']:
        if row['answer'].lower() == 'yes':
            return -3
        elif row['answer'].lower() == 'no':
            return 3
    elif row['question'] in ['T1_S2_IN1', 'T1_S2_IN2']:
        if row['answer'].lower() == 'no':
            return -3
    return 0

mergedAnswers_df['penalty_bonus'] = mergedAnswers_df.apply(apply_penalty_bonus, axis=1)

mergedAnswers_df['score'] = mergedAnswers_df['is_correct'].astype(int) + mergedAnswers_df['penalty_bonus']

In [ ]:
correct_count = mergedAnswers_df['score'].sum()
print(f"Number of correct answers: {correct_count}")

correct_by_version = mergedAnswers_df.groupby('version')['score'].sum()
print("\nCorrect answers by version:")
print(correct_by_version)

total_by_version = mergedAnswers_df.groupby('version')['score'].count()
incorrect_by_version = total_by_version - correct_by_version
print("\nIncorrect answers by version:")
print(incorrect_by_version)

percent_accuracy = correct_by_version/total_by_version
print("\nPercent accuracy by version:")
print(percent_accuracy)

answers_df_filtered = answers_df[answers_df['version'] != 'undefined']

# Calculate time differences (speed) between consecutive questions
answers_df_filtered['time_diff'] = answers_df_filtered['time'].diff().dt.total_seconds()

# Group by 'version' and calculate average speed
grouped = answers_df_filtered.groupby('version')['time_diff']
mean_speed_by_version = grouped.mean()
std_speed_by_version = grouped.std()

Q1 = grouped.quantile(0.25)
Q3 = grouped.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = {version: answers_df_filtered[(answers_df_filtered['version'] == version) & 
                                ((answers_df_filtered['time_diff'] < lower_bound[version]) | 
                                 (answers_df_filtered['time_diff'] > upper_bound[version]))]['time_diff']
            for version in answers_df_filtered['version'].unique()}

mean_speed_by_version.plot(kind='bar', yerr=std_speed_by_version, capsize=4, color='skyblue', label='Average Time Difference')

# Add outliers as scatter points
for version, outlier_values in outliers.items():
    x = mean_speed_by_version.index.get_loc(version)
    plt.scatter([x]*len(outlier_values), outlier_values, color='red', zorder=5, label='Outliers' if x == 0 else "")


# Plotting
plt.title('Average Time Spent on Questions by Version with Error Bars and Outliers')
plt.xlabel('Version')
plt.ylabel('Average Time Difference (seconds)')
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# correctness separated by phase
t1_s1_df = mergedAnswers_df[mergedAnswers_df['pageName'].str.startswith('T1_S1')]
t1_s2_df = mergedAnswers_df[mergedAnswers_df['pageName'].str.startswith('T1_S2')]

# Calculate the correctness for T1_S1 and T1_S2
correct_by_t1_s1 = t1_s1_df['is_correct'].sum()
total_by_t1_s1 = t1_s1_df['is_correct'].count()
correct_by_t1_s2 = t1_s2_df['is_correct'].sum()
total_by_t1_s2 = t1_s2_df['is_correct'].count()

# Create a DataFrame for plotting
correctness_df = pd.DataFrame({
    'Correct': [correct_by_t1_s1, correct_by_t1_s2],
    'Total': [total_by_t1_s1, total_by_t1_s2]
}, index=['T1_S1', 'T1_S2'])

# Calculate the percentage of correct answers
correctness_df['Percentage'] = (correctness_df['Correct'] / correctness_df['Total']) * 100

# Plotting the results
plt.figure(figsize=(8, 6))
correctness_df['Percentage'].plot(kind='bar', color=['skyblue', 'lightgreen'], edgecolor='black')
plt.title('Correctness of Answers for T1_S1 and T1_S2')
plt.xlabel('Page Name Prefix')
plt.ylabel('Percentage of Correct Answers')
plt.xticks(rotation=0)
plt.ylim(0, 100)  # Ensuring the y-axis goes from 0 to 100%
plt.grid(axis='y')
plt.tight_layout()
plt.show()